In [1]:
!pip install tweepy

In [2]:
!pip install openpyxl

In [26]:
import numpy as np
import pandas as pd
import tweepy
import json
from tweepy import OAuthHandler # responsible for communicating with twitter

In [4]:
consumer_key_api = "RTrWYjZgpLgNTL6idGiH1I6RT"
consumer_secret_api = "PV8lvg5DJLp7fMsp83LErAqMOpPiIqHE3b11cJQ7BgTe1k9miR"
access_token = "1321760436207919105-gLwHRd3Fo50DKQ32vUKZhp8tyfvwdU"
access_token_secret = "qydEY4MwMsLRybAA21vMI4ScArCJ214wTRIK7rwnnyyHW"

In [5]:
df = pd.DataFrame(columns = ['Tweets' , 'User' , 'User_statuses_count' , 
                            'user_followers' , 'User_location' , 'User_verified' ,
                            'fav_count' , 'rt_count' , 'tweet_date'] )
print(df)
print(df.shape)

Empty DataFrame
Columns: [Tweets, User, User_statuses_count, user_followers, User_location, User_verified, fav_count, rt_count, tweet_date]
Index: []
(0, 9)


In [8]:
auth = tweepy.OAuthHandler( consumer_key_api , consumer_secret_api )
auth.set_access_token( access_token , access_token_secret )
api = tweepy.API(auth)

In [9]:
def stream(data, file_name):
    i = 0
    for tweet in tweepy.Cursor(api.search, q=data, count=100, lang='en').items():
        print(i, end='\r')
        df.loc[i, 'Tweets'] = tweet.text
        df.loc[i, 'User'] = tweet.user.name
        df.loc[i, 'User_statuses_count'] = tweet.user.statuses_count
        df.loc[i, 'user_followers'] = tweet.user.followers_count
        df.loc[i, 'User_location'] = tweet.user.location
        df.loc[i, 'User_verified'] = tweet.user.verified
        df.loc[i, 'fav_count'] = tweet.favorite_count
        df.loc[i, 'rt_count'] = tweet.retweet_count
        df.loc[i, 'tweet_date'] = tweet.created_at
        df.to_excel('{}.xlsx'.format(file_name))
        i = i+1
        if i == 1000:
            break
        else:
            pass

In [10]:
stream(data=["DC winning IPL"] , file_name='my_tweets')

In [11]:
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date
0,This time I badly want to see DC winning the I...,𝔾𝕒𝕦𝕣𝕒𝕧,8494,146,"New Delhi, India",False,0,0,2020-11-02 17:28:22
1,RT @ayshahabib11: All IPL fans what do you thi...,#aisihihunmein,17051,348,,False,0,42,2020-11-02 17:27:00
2,RT @FirstpostSports: Rishabh Pant hits the win...,Firstpost,611563,2088559,"Mumbai, India",True,0,1,2020-11-02 17:25:56
3,Rishabh Pant hits the winning runs. \n\nDelhi ...,Firstpost Sports,208492,441729,India,True,17,1,2020-11-02 17:25:39
4,RT @ayshahabib11: All IPL fans what do you thi...,सिया_उर्मिल 🇮🇳MSDIAN💛,30745,457,,False,0,42,2020-11-02 17:15:55


In [14]:
!pip install textblob  # inbuild sentiment analyser

In [16]:
from textblob import TextBlob

In [17]:
import re
def clean_tweet(tweet):
    return ' '.join(re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', ' ', tweet).split())

In [21]:
df['clean_tweet'] = df['Tweets'].apply(lambda x : clean_tweet(x))

In [22]:
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date,clean_tweet
0,This time I badly want to see DC winning the I...,𝔾𝕒𝕦𝕣𝕒𝕧,8494,146,"New Delhi, India",False,0,0,2020-11-02 17:28:22,This time I badly want to see DC winning the I...
1,RT @ayshahabib11: All IPL fans what do you thi...,#aisihihunmein,17051,348,,False,0,42,2020-11-02 17:27:00,RT All IPL fans what do you think who s winnin...
2,RT @FirstpostSports: Rishabh Pant hits the win...,Firstpost,611563,2088559,"Mumbai, India",True,0,1,2020-11-02 17:25:56,RT Rishabh Pant hits the winning runs Delhi Ca...
3,Rishabh Pant hits the winning runs. \n\nDelhi ...,Firstpost Sports,208492,441729,India,True,17,1,2020-11-02 17:25:39,Rishabh Pant hits the winning runs Delhi Capit...
4,RT @ayshahabib11: All IPL fans what do you thi...,सिया_उर्मिल 🇮🇳MSDIAN💛,30745,457,,False,0,42,2020-11-02 17:15:55,RT All IPL fans what do you think who s winnin...


In [23]:
def analyze_sentiment(tweet):
    analysis = TextBlob(tweet)
    if analysis.sentiment.polarity > 0:
        return 'Positive'
    elif analysis.sentiment.polarity == 0:
        return 'Neutral'
    else:
        return 'Negative'

In [24]:
df['Sentiment'] = df['clean_tweet'].apply(lambda x : analyze_sentiment(x) )
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date,clean_tweet,Sentiment
0,This time I badly want to see DC winning the I...,𝔾𝕒𝕦𝕣𝕒𝕧,8494,146,"New Delhi, India",False,0,0,2020-11-02 17:28:22,This time I badly want to see DC winning the I...,Negative
1,RT @ayshahabib11: All IPL fans what do you thi...,#aisihihunmein,17051,348,,False,0,42,2020-11-02 17:27:00,RT All IPL fans what do you think who s winnin...,Positive
2,RT @FirstpostSports: Rishabh Pant hits the win...,Firstpost,611563,2088559,"Mumbai, India",True,0,1,2020-11-02 17:25:56,RT Rishabh Pant hits the winning runs Delhi Ca...,Positive
3,Rishabh Pant hits the winning runs. \n\nDelhi ...,Firstpost Sports,208492,441729,India,True,17,1,2020-11-02 17:25:39,Rishabh Pant hits the winning runs Delhi Capit...,Positive
4,RT @ayshahabib11: All IPL fans what do you thi...,सिया_उर्मिल 🇮🇳MSDIAN💛,30745,457,,False,0,42,2020-11-02 17:15:55,RT All IPL fans what do you think who s winnin...,Positive


In [25]:
n = 1
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 RT @ayshahabib11: All IPL fans what do you think who's winning the match today and thus qualifying for the playoffs ? 

DC or RCB ? 

Comme…

Clean tweet:
 RT All IPL fans what do you think who s winning the match today and thus qualifying for the playoffs DC or RCB Comme

Sentiment of the tweet:
 Positive
